In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import time
from PMTK.sampling.preferences_sampler import *
from PMTK.pref.preferences import *
from PMTK.utility.utility_solver import *
from PMTK.sampling.subset_samplers import *
from PMTK.utility.subset_finder import *
from PMTK.sampling.gibbs import *
from PMTK.data.film_dataset import *
from PMTK.sampling.decider import *
from PMTK.utility import *

In [8]:
def build_approx_theta(prf, init_theta = None):
    connivents = []
    if not init_theta:
        init_theta = [EMPTY_SET]
    theta = init_theta
    min_k = 1
    c  = get_connivent(theta, prf)
    cpt = 0
    while c:
        if not c in connivents:
            connivents.append(c)
        cit = get_candidate_iterator(c)
        skey = sorted(cit.keys())[0]
        b = False
        for k in cit:
            if b:
                break
            for i in cit[k]:
                s = set(i)
                for t in s:
                    b = b or (check_connivence_resolution(c, t) and not t in theta)
                    if not t in theta and check_connivence_resolution(c, t):
                        theta.append(t)
        c  = get_connivent(theta, prf)
        cpt = cpt + 1
        #print("solved connivent: ", cpt, " with", theta)
    a = additivity(theta)
    for c_i in connivents:
        cit = get_candidate_iterator(c_i)
        for k in cit:
            if k > a:
                break
            for i in cit[k]:
                for t in i:
                    if not t in theta and check_connivence_resolution(c_i,t):
                        theta.append(t)
    
    return theta

In [9]:
def predict_from_mult_thetas(thetas,p, subsets):
    mdls = [utility_polyhedron(p.items, t, p) for t in thetas]
    prf = Preferences(p.items)
    for i_1 in range(len(subsets)):
        s_1 = subsets[i_1]
        for i_2 in range(i_1 + 1, len(subsets)):
            s_2 = subsets[i_2]
            r = [ordinal_dominance(s_1, s_2, mdl) for mdl in mdls]
            if all(i == "SUP" for i in r):
                prf.add_preference(s_1, s_2)
            elif all(i == "INF" for i in r):
                prf.add_preference(s_2, s_1)
            elif all(i == "EQ" for i in r):
                prf.add_indifference(s_1,s_2)
    return prf

def predict_from_theta(theta, p, subsets):
    mdl = utility_polyhedron(p.items, theta, p)
    prf = ordinal_peferences(p.items, subsets, mdl)
    return prf

def complete_theta(theta):
    t2 = []
    for x in theta:
        for s in get_all_k_sets(x, len(x)):
            if not s in t2:
                t2.append(s)
    return t2

In [10]:
def ACR(prf, decider):
    C = 0
    W = 0
    T = len(prf.preferred)
    for x,y in prf.preferred:
        if decider(x) > decider(y):
            C += 1
        elif decider(x) < decider(y):
            W += 1
    return C,W,T

def intersect_per_union(prf_1, prf_2):
    intersection = prf_1.intersection(prf_2)
    un = prf_1 + prf_2
    return len(intersection) / len(un)

In [11]:
n_items = 6
items = np.arange(n_items)
ground_truth = Tierlist_Decider(items, p=0.3, alpha = 0.3)
decider = Objective_Function(items, ground_truth)

budget = 25
n_subsets = 10


ground_truth = Tierlist_Decider(items, p=0.3, alpha = 0.3)
decider = Objective_Function(items, ground_truth)

for k in range(10):
    for b in range(budget):
        s = sample_subset(items)
        while s in decider.saved:
            s = sample_subset(items)
        decider(s)
        prf = decider.relation()
        if len(prf) < 5:
            continue
            
        t_heuristic = build_approx_theta(prf, [EMPTY_SET])
        #t_lex2 = get_kernels_lex2(prf, get_all_k_sets(items, len(items)))
        #t_lex3 = get_kernels_lex3(prf, get_all_k_sets(items, len(items)))
        print("%%%%%%%%% Computting all vars %%%%%%%%")
        t_var = get_kernels_var(prf, get_all_k_sets(items, len(items)))
        print(f"%%%%%%%%% Computed = {t_var} %%%%%%%%", end = "\n\n")
    
        test_subsets = sample_subsets(items, n_subsets=n_subsets)
      
        #prf_t_heuristic = predict_from_theta(t_heuristic, prf, test_subsets)
        #prf_tlex2_all = predict_from_mult_thetas(t_lex2, prf, test_subsets)
        #prf_tlex3_all = predict_from_mult_thetas(t_lex3, prf, test_subsets)
        prf_var_all = predict_from_mult_thetas(t_var, prf, test_subsets)
    

%%%%%%%%% Computting all vars %%%%%%%%
%%%%%%%%% Computed = [[(3,), (4,)], [(1,), (3,)]] %%%%%%%%

%%%%%%%%% Computting all vars %%%%%%%%
%%%%%%%%% Computed = [[(2,), (4,), (5,)], [(1,), (2,), (5,)], [(2,), (3,), (5,)], [(1,), (3,), (4,)]] %%%%%%%%



KeyboardInterrupt: 

## Listes des $\theta$ expérimentés:

- Union des $\theta$ qui minimisent lexicographiquement (additivité, taille).
- Un des $\theta$ qui minimisent lexicographiquement (additivité, taille).

- Union des $\theta$ qui minimisent lexicographiquement (additivité, taille, somme des tailles).
- Un des $\theta$ qui minimisent lexicographiquement (additivité, taille, somme des tailles).

- Union des $\theta$ qui minimisent l'additivité.
- Un des $\theta$ qui minimisent l'additivité.

- Union des $\theta$ qui minimisent la variance.
- Un des $\theta$ qui minimisent la variance.


In [12]:
n_items = 5
items = np.arange(n_items)
ground_truth = Tierlist_Decider(items, p=0.3, alpha = 0.3)
decider = Objective_Function(items, ground_truth)

budget = 25
n_subsets = 10

data = {
    "budget":[],
    "theta_definition":[],
    "theta_operator":[],
    "n_theta_min":[],
    
    "n_preferences":[],
    "additivity":[],
    "size":[],
    "sizes_sum":[],
    "time":[],
    "C":[],
    "W":[],
    "T":[],
}

contradictions_data = {
    "theta_1":[],
    "theta_2":[],
    "union_per_intersection":[]
}

for repetition in range(10):
    ground_truth = Tierlist_Decider(items, p=0.3, alpha = 0.3)
    decider = Objective_Function(items, ground_truth)

    for b in range(budget):
        s = sample_subset(items)
        while s in decider.saved:
            s = sample_subset(items)
            
        decider(s)
        prf = decider.relation()
        if len(prf) < 5:
            continue
            
        t_heuristic = build_approx_theta(prf, [EMPTY_SET])
            
        ti_lex_2 = time.time()
        t_lex2 = get_kernels_lex2(prf, get_all_k_sets(items, len(items)))
        ti_lex_2 = time.time() - ti_lex_2
        
        t_lex2_rnd = random.choice(t_lex2)
        t_lex2_union = union(t_lex2)
        
        
        ti_lex_3 = time.time()
        t_lex3 = get_kernels_lex3(prf, get_all_k_sets(items, len(items)))
        ti_lex_3 = time.time() - ti_lex_3
        
        t_lex3_rnd = random.choice(t_lex3)
        t_lex3_union = union(t_lex3)
        
        
        ti_var = time.time()
        t_var = get_kernels_var(prf, get_all_k_sets(items, len(items)))
        ti_var = time.time() - ti_var
        
        t_var_rnd = random.choice(t_var)
        t_var_union = union(t_var)
        
        print(f"Lex2 took {ti_lex_2:.2f}")
        print(f"Lex3 took {ti_lex_3:.2f}")
        print(f"Var took {ti_var:.2f}")


        for s_rep in range(10):
            print(f"repetition = {repetition}, b = {b}, srep = {s_rep}, npref = {len(prf)}")
            test_subsets = sample_subsets(items, n_subsets=n_subsets)
            
            

            #print(f"t_union = {t_union}, t_heuristic = {t_heuristic}, t_mins = {t_mins}, t_random = {t_random}")
            
            prf_t_heuristic = predict_from_theta(t_heuristic, prf, test_subsets)

            
            prf_tlex2_union = predict_from_theta(t_lex2_union, prf, test_subsets)
            prf_tlex2_rnd = predict_from_theta(t_lex2_rnd, prf, test_subsets)
            prf_tlex2_all = predict_from_mult_thetas(t_lex2, prf, test_subsets)
            
            prf_tlex3_union = predict_from_theta(t_lex3_union, prf, test_subsets)
            prf_tlex3_rnd = predict_from_theta(t_lex3_rnd, prf, test_subsets)
            prf_tlex3_all = predict_from_mult_thetas(t_lex3, prf, test_subsets)
            
            prf_var_union = predict_from_theta(t_var_union, prf, test_subsets)
            prf_var_rnd = predict_from_theta(t_var_rnd, prf, test_subsets)
            prf_var_all = predict_from_mult_thetas(t_var[:10], prf, test_subsets)

            
            C_t_heuristic, W_t_heuritistic, T_t_heuristic = ACR(prf_t_heuristic, ground_truth)
            
            C_tlex2_union, W_tlex2_union, T_tlex2_union = ACR(prf_tlex2_union, ground_truth)
            C_tlex2_rnd, W_tlex2_rnd, T_tlex2_rnd = ACR(prf_tlex2_rnd, ground_truth)
            C_tlex2_all, W_tlex2_all, T_tlex2_all = ACR(prf_tlex2_all, ground_truth)
            
            C_tlex3_union, W_tlex3_union, T_tlex3_union = ACR(prf_tlex3_union, ground_truth)
            C_tlex3_rnd, W_tlex3_rnd, T_tlex3_rnd = ACR(prf_tlex3_rnd, ground_truth)
            C_tlex3_all, W_tlex3_all, T_tlex3_all = ACR(prf_tlex3_all, ground_truth)
            
            C_var_union, W_var_union, T_var_union = ACR(prf_var_union, ground_truth)
            C_var_rnd, W_var_rnd, T_var_rnd = ACR(prf_var_rnd, ground_truth)
            C_var_all, W_var_all, T_var_all = ACR(prf_var_all, ground_truth)
            
            ###LEX 2
            data["budget"].append(b) 
            data["theta_definition"].append("LEX2")
            data["theta_operator"].append("UNION")
            data["time"].append(ti_lex_2)
            data["n_theta_min"].append(len(t_lex2))
            data["n_preferences"].append(len(prf))
            data["additivity"].append(additivity(t_lex2_union))
            data["size"].append(len(t_lex2_union))
            data["sizes_sum"].append(sum(len(i) for i in t_lex2_union))
            data["C"].append(C_tlex2_union)
            data["W"].append(W_tlex2_union)
            data["T"].append(T_tlex2_union)
            
            data["budget"].append(b) 
            data["theta_definition"].append("LEX2")
            data["theta_operator"].append("RND")
            data["time"].append(ti_lex_2)
            data["n_theta_min"].append(len(t_lex2))
            data["n_preferences"].append(len(prf))
            data["additivity"].append(additivity(t_lex2_rnd))
            data["size"].append(len(t_lex2_rnd))
            data["sizes_sum"].append(sum(len(i) for i in t_lex2_rnd))
            data["C"].append(C_tlex2_rnd)
            data["W"].append(W_tlex2_rnd)
            data["T"].append(T_tlex2_rnd)
            
            data["budget"].append(b) 
            data["theta_definition"].append("LEX2")
            data["theta_operator"].append("ALL")
            data["time"].append(ti_lex_2)
            data["n_theta_min"].append(len(t_lex2))
            data["n_preferences"].append(len(prf))
            data["additivity"].append(additivity(t_lex2[0]))
            data["size"].append(len(t_lex2[0]))
            data["sizes_sum"].append(sum(len(i) for i in t_lex2[0]))
            data["C"].append(C_tlex2_all)
            data["W"].append(W_tlex2_all)
            data["T"].append(T_tlex2_all)
            
            
            ####LEX 3 
            data["budget"].append(b) 
            data["theta_definition"].append("LEX3")
            data["theta_operator"].append("UNION")
            data["time"].append(ti_lex_3)
            data["n_theta_min"].append(len(t_lex3))
            data["n_preferences"].append(len(prf))
            data["additivity"].append(additivity(t_lex3_union))
            data["size"].append(len(t_lex3_union))
            data["sizes_sum"].append(sum(len(i) for i in t_lex3_union))
            data["C"].append(C_tlex3_union)
            data["W"].append(W_tlex3_union)
            data["T"].append(T_tlex3_union)
            
            data["budget"].append(b) 
            data["theta_definition"].append("LEX3")
            data["theta_operator"].append("RND")
            data["time"].append(ti_lex_3)
            data["n_theta_min"].append(len(t_lex3))
            data["n_preferences"].append(len(prf))
            data["additivity"].append(additivity(t_lex3_rnd))
            data["size"].append(len(t_lex3_rnd))
            data["sizes_sum"].append(sum(len(i) for i in t_lex3_rnd))
            data["C"].append(C_tlex3_rnd)
            data["W"].append(W_tlex3_rnd)
            data["T"].append(T_tlex3_rnd)
            
            data["budget"].append(b) 
            data["theta_definition"].append("LEX3")
            data["theta_operator"].append("ALL")
            data["time"].append(ti_lex_3)
            data["n_theta_min"].append(len(t_lex3))
            data["n_preferences"].append(len(prf))
            data["additivity"].append(additivity(t_lex3[0]))
            data["size"].append(len(t_lex3[0]))
            data["sizes_sum"].append(sum(len(i) for i in t_lex3[0]))
            data["C"].append(C_tlex3_all)
            data["W"].append(W_tlex3_all)
            data["T"].append(T_tlex3_all)
            
            
            ####VARIANCE  
            data["budget"].append(b) 
            data["theta_definition"].append("VAR")
            data["theta_operator"].append("UNION")
            data["time"].append(ti_var)
            data["n_theta_min"].append(len(t_var))
            data["n_preferences"].append(len(prf))
            data["additivity"].append(additivity(t_var_union))
            data["size"].append(len(t_var_union))
            data["sizes_sum"].append(sum(len(i) for i in t_var_union))
            data["C"].append(C_var_union)
            data["W"].append(W_var_union)
            data["T"].append(T_var_union)
            
            data["budget"].append(b) 
            data["theta_definition"].append("VAR")
            data["theta_operator"].append("RND")
            data["time"].append(ti_var)
            data["n_theta_min"].append(len(t_var))
            data["n_preferences"].append(len(prf))
            data["additivity"].append(additivity(t_var_rnd))
            data["size"].append(len(t_var_rnd))
            data["sizes_sum"].append(sum(len(i) for i in t_var_rnd))
            data["C"].append(C_var_rnd)
            data["W"].append(W_var_rnd)
            data["T"].append(T_var_rnd)
            
            data["budget"].append(b) 
            data["theta_definition"].append("VAR")
            data["theta_operator"].append("ALL")
            data["time"].append(ti_var)
            data["n_theta_min"].append(len(t_var))
            data["n_preferences"].append(len(prf))
            data["additivity"].append(additivity(t_var[0]))
            data["size"].append(len(t_var[0]))
            data["sizes_sum"].append(sum(len(i) for i in t_var[0]))
            data["C"].append(C_var_all)
            data["W"].append(W_var_all)
            data["T"].append(T_var_all)
            
            df = pd.DataFrame(data)
            df.to_csv("theta_definitions_comparisons_3.csv")
            #print(df)
        
        
        
    

Lex2 took 0.16
Lex3 took 0.18
Var took 0.14
repetition = 0, b = 3, srep = 0, npref = 5
repetition = 0, b = 3, srep = 1, npref = 5
repetition = 0, b = 3, srep = 2, npref = 5
repetition = 0, b = 3, srep = 3, npref = 5
repetition = 0, b = 3, srep = 4, npref = 5
repetition = 0, b = 3, srep = 5, npref = 5
repetition = 0, b = 3, srep = 6, npref = 5
repetition = 0, b = 3, srep = 7, npref = 5
repetition = 0, b = 3, srep = 8, npref = 5
repetition = 0, b = 3, srep = 9, npref = 5
Lex2 took 0.18
Lex3 took 0.18
Var took 0.13
repetition = 0, b = 4, srep = 0, npref = 8
repetition = 0, b = 4, srep = 1, npref = 8
repetition = 0, b = 4, srep = 2, npref = 8
repetition = 0, b = 4, srep = 3, npref = 8
repetition = 0, b = 4, srep = 4, npref = 8
repetition = 0, b = 4, srep = 5, npref = 8
repetition = 0, b = 4, srep = 6, npref = 8
repetition = 0, b = 4, srep = 7, npref = 8
repetition = 0, b = 4, srep = 8, npref = 8
repetition = 0, b = 4, srep = 9, npref = 8
Lex2 took 0.27
Lex3 took 0.19
Var took 0.14
repetiti